# Importing necessary libraries

In [1]:
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
import actionlib_msgs.msg
from actionlib_msgs.msg import GoalStatus
import assignment_2_2023.msg
from assignment_2_2023.msg import Vel


import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
from IPython.display import display 
from IPython.display import clear_output

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tf.transformations import quaternion_matrix

#initialize the publisher to publish position and velocity
pub = rospy.Publisher("/kinematics" , Vel , queue_size = 1) 

have_goal = False

Goals_given_x = []
Goals_given_y = []

Goals_cancelled_x = []
Goals_cancelled_y = []

current_x = 0
current_y = 0

x_data = []
y_data = []


# Define the callback function for the subscriber 

In [2]:
def publish_kinematics(msg):
    
    global pub, current_x, current_y, x_data, y_data
    
    # get the position and velocity of the robot from Odometry
    
    linear_vel_now = msg.twist.twist.linear
    angular_vel_now = msg.twist.twist.angular
    
    current_x = msg.pose.pose.position.x
    current_y = msg.pose.pose.position.y
    
    x_data.append(msg.pose.pose.position.x)
    y_data.append(msg.pose.pose.position.y)
    
    # create a new position and velocity message
    pos_vel = Vel()
    pos_vel.x = msg.pose.pose.position.x
    pos_vel.y = msg.pose.pose.position.y
    pos_vel.vel_x = linear_vel_now.x
    pos_vel.vel_z = angular_vel_now.z
    
    #Publish the custom message
    pub.publish(pos_vel)

# Define cancel button functionality


In [3]:
def on_cancel_button_clicked(b):
    global have_goal, client, Goals_cancelled_x, Goals_cancelled_y, Goals_reached_no 
    
   
    
    with output:
        
        if have_goal:
            
            
                
            Goals_cancelled_x.append(rospy.get_param('/des_pos_x'))
            Goals_cancelled_y.append(rospy.get_param('/des_pos_y'))
            
                        
            # Cancel the goal
            have_goal = False
            client.cancel_goal()
            rospy.loginfo("The goal is cancelled successfully")
            print("Target cancelled")
                           
            
            
        elif not have_goal:
            
            rospy.logwarn("There are no goals to be cancelled. First determine a goal!!!")
            
             
            

# define change button functionality 

In [4]:
def on_change_button_clicked(b):
    
    global have_goal, goal_new, client, Goals_given_x, Goals_given_y
    
    
    
    with output:
        
        if have_goal:
            
            rospy.logwarn('You should first stop the current target')
            
            
        else:
            
            
            # get the value for x and y targets
            
            rospy.set_param('/des_pos_x' , c1_slider.value)
            rospy.set_param('/des_pos_y' , c2_slider.value)
            
            x_target = c1_slider.value
            y_target = c2_slider.value
            
            goal_new = assignment_2_2023.msg.PlanningGoal()
            goal_new.target_pose.pose.position.x = x_target
            goal_new.target_pose.pose.position.y = y_target
            
            
            # saving goals in a list
            Goals_given_x.append(x_target)
            Goals_given_y.append(y_target)
            
            #
            
            client.send_goal(goal_new)
            have_goal = True

        
            print("New target set: X =",  goal_new.target_pose.pose.position.x, "Y =", goal_new.target_pose.pose.position.y)
            
             

# define info button functionality 

In [5]:
def on_Info_button_clicked(b):
    
    global Goals_given_x, Goals_given_y, Goals_cancelled_x, Goals_cancelled_y, current_x, current_y
    
    
    
    
    with output:
        
        
                
        print ('current position is', 'X =', current_x, 'Y =', current_y)
        
        target_list = list(zip(Goals_given_x, Goals_given_y))
        print('The goals are: \n', target_list)
        
        cancel_list = list(zip(Goals_cancelled_x, Goals_cancelled_y))
        print('The cancelled goals are:\n', cancel_list)
            
             

# Path Visualizer 

In [6]:
class Visualiser:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()  
        self.ln, = self.ax.plot([], [], 'ro', label='Robot Position')  # Target's Position Plot
        self.goal_ln, = self.ax.plot([], [], 'bo', markersize=10, label='Robot Goal Position')  # Robot's Position Data Arrays
        self.cancel_goal_ln, = self.ax.plot([], [], 'rx', markersize=10, label='Robot Cancelled Goal Position')  # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        self.Goal_x, self.Goal_y = [], []
        
        
        
    def plot_init(self):
        self.ax.set_title("Robot Position", fontsize=15, fontweight='bold')  # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize=10, fontweight="bold")
        self.ax.set_ylabel("Y [m]", fontsize=10, fontweight="bold")
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.grid(True)
        self.ax.legend()
        

        return self.ln, self.goal_ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        x_t = rospy.get_param('/des_pos_x')
        y_t = rospy.get_param('/des_pos_y')
        self.Goal_x.append(x_t)
        self.Goal_y.append(y_t)
        self.cancelled_x = np.array(Goals_cancelled_x)
        self.cancelled_y = np.array(Goals_cancelled_y)
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.goal_ln.set_data(self.Goal_x, self.Goal_y)
        self.cancel_goal_ln.set_data(self.cancelled_x, self.cancelled_y)
        
        
        return self.ln, self.goal_ln, self.cancel_goal_ln


# Reached Goal visualizer

In [7]:
class Goal_Visualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        # Set Plot Title
        self.ax.set_title("Reached & Cancelled Goals", fontsize = 20, fontweight = "bold")
        
        self.ax.grid(axis = 'y', color = 'grey', linestyle = '-', alpha = 0.5)
        # Set Plot Labels on x-axis
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 10])
        self.ax.set_yticks(np.arange(0, 11, 1))
        self.bar_colors = ['blue', 'red']
        self.bar_plot = self.ax.bar(self.x_pos, [self.reached, self.cancelled], align = 'center', color = self.bar_colors, width = 0.2)

    def goal_callback(self, msg):
        # Calculate the number of reached/cancel goals
        if msg.status.status == 3:
            self.reached += 1
        elif msg.status.status == 2:
            self.cancelled += 1 

    def update_plot(self, frame):
        
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
            bar.set_color(self.bar_colors[i])
            
        return self.bar_plot

# Initialize the node and subscriber to the odom topic 

In [8]:
rospy.init_node('node_a_client')

rospy.loginfo('node a started successully')


# initialize action client 
client  = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)

client.wait_for_server()

rospy.loginfo("Action successfully initialized.")


    
#subscribe to odom topic and publish the obtained position and velocity on the custom message we created
    
rospy.Subscriber("/odom", Odometry , publish_kinematics)
    
# Get the current target from the parameters in the launch file 
    
# In this case it's 0 and 1
    
x_t = rospy.get_param('/des_pos_x')
y_t = rospy.get_param('/des_pos_y')
    
rospy.loginfo("Current goal is: target_x = %f, target_y = %f", x_t , y_t )
    





[INFO] [1716035821.857657, 0.000000]: node a started successully
[INFO] [1716035822.196620, 1225.874000]: Action successfully initialized.
[INFO] [1716035822.236295, 1225.888000]: Current goal is: target_x = 0.000000, target_y = 1.000000


# defining target slider 

In [9]:
# integer slider for x_target

c1_slider = widgets.IntSlider(value = 1,
                      min = -10 , 
                      max = 10,
                      step = 1,
                      description = 'X_target',
                      disabled = False,
                      continuous_update = False , 
                      orientation = 'horizontal',
                      readout = True,
                      readout_format = 'd')

# integer slider for y_target

c2_slider = widgets.IntSlider(value = 0,
                      min = -10 , 
                      max = 10,
                      step = 1,
                      description = 'Y_target',
                      disabled = False,
                      continuous_update = False , 
                      orientation = 'horizontal',
                      readout = True,
                      readout_format = 'd')

instructions_html1 = """
<h3 style="color: blue;">Choose the target here:</h3>
"""
    
# Create HTML widget with instructions
instructions_widget1 = widgets.HTML(value=instructions_html1)


display(instructions_widget1, c1_slider,c2_slider)

HTML(value='\n<h3 style="color: blue;">Choose the target here:</h3>\n')

IntSlider(value=1, continuous_update=False, description='X_target', max=10, min=-10)

IntSlider(value=0, continuous_update=False, description='Y_target', max=10, min=-10)

# Button and widget description 

In [10]:
   
instructions_html2 = """
<h3 style="color: blue;">Instructions:</h3>
<p style = "font-size: 16px;">1. Choose what you want to do next:</p>
<p style = "font-size: 16px;">2. Choose Change to change the target.</p>
<p style = "font-size: 16px;">3. Choose Change to change the target.(determine the target in the slider above the page before choosing Change)</p>
<p style="color: red; font-family: 'Times New Roman', serif; font-size: 18px;"> Note:  Keep in mind that you should cancel the movement before setting a new target or you will get a warning (Even if the target is reached, you need to push cancel button once before setting a new target.).</p>
"""
    
# Create HTML widget with instructions
instructions_widget2 = widgets.HTML(value=instructions_html2)
    


    
# ask user if they want to cancel the goal or move towards a new goal
instructions_html3 = """
<p style = "font-size: 22px;">What's your choice now:</p>
"""
    
# Create HTML widget with instructions
instructions_widget3= widgets.HTML(value=instructions_html3)
    


    
# Choice maker 
    
# from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
    
b1 = Button(description='Change',
            layout=Layout(width='auto', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='green'))
    
b2 = Button(description='Cancel',
            layout=Layout(width='auto', grid_area='b2'),
            style=ButtonStyle(button_color='red'))

b3 = Button(description='General Info',
            layout=Layout(width='auto', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='lightblue'))
    
output = widgets.Output()
    
b1.on_click(on_change_button_clicked)
b2.on_click(on_cancel_button_clicked)
b3.on_click(on_Info_button_clicked)
    
buttons_box = VBox([HBox([b3]),HBox([b1,b2])])
output_box = VBox([buttons_box, output])
display(instructions_widget2 , instructions_widget3 , output_box)

HTML(value='\n<h3 style="color: blue;">Instructions:</h3>\n<p style = "font-size: 16px;">1. Choose what you wa…

HTML(value='\n<p style = "font-size: 22px;">What\'s your choice now:</p>\n')

# Online plot 

In [11]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)


visualize_goals = Goal_Visualizer()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2023.msg.PlanningActionResult, visualize_goals.goal_callback)

animation_goal = FuncAnimation(visualize_goals.fig, visualize_goals.update_plot, interval = 1000)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …